In [1]:
%config IPCompleter.greedy=True

# Creación de Catalogo

Se tomara un archivo de excel previamente tratado y se guardara en una base de datos<br>
El Archivo tendra las siguientes columnas:
<ol>
    <li>Nombre.</li>
    <li>Latitud(en formato decimal)</li>
    <li>Lonigtud(en formato decimal)</li>
</ol>

In [18]:
#Bibliotecas que se usaran.
import pandas as pd
import glob
import pyodbc
import sqlalchemy

In [3]:
#Funcion que nos dara todas rutas de los archivos contenidas dentro de una ruta especificada.
from os import scandir, getcwd

def ls(ruta = getcwd()):
    return [arch.name for arch in scandir(ruta) if arch.is_file()]

In [39]:
#Leemos el archivo excel que contiene la tabla catalogo
df = pd.read_excel("Catalogo.xlsx")
#Nos deshacemos la columna index que del dataframe que tambien se guardo en el excel.
df = df.drop(['Unnamed: 0'], axis=1)
#Se crea un nuevo dataframe con los id a usar y se agregan las columnas indicadas con los nuevos nombres.
catalogo = pd.DataFrame({'id': list(range(1,df.size))})
catalogo[['nombre','latitud','longitud']] = df[['Nombre Completo','Latitud N', 'Longitud W']]

In [40]:
catalogo.head()

,id,nombre,latitud,longitud
0,1,"CALVILLO, AGUASCALIENTES",21.849444,-102.712222
1,2,"PRESA 50 ANIV. , AGUASCALIENTES",22.189306,-102.464972
2,3,"SIERRA FRIA, AGUASCALIENTES",22.269778,-102.608778
3,4,"BAHÍA DE LOS ANGELES, BAJA CALIFORNIA",28.896389,-113.560278
4,5,"CATAVIÑA, BAJA CALIFORNIA",29.727222,-114.719167


In [11]:
#Se crea una ruta de conexion a la base de datos llamada clima, la cual se encuentra en el servidor DESKTOP-HQI9VLS
#Utilizamos la autentificacion de windows para conectarnos a este.
server = 'DESKTOP-HQI9VLS'
database = 'clima'
engine = sqlalchemy.create_engine('mssql+pyodbc://'+server+'/'+database+'?trusted_connection=yes&driver=SQL+Server')

In [50]:
#importamos los tipos para no tener para facilitar su uso
from sqlalchemy.types import VARCHAR,Float,Integer

In [51]:
#Utilizando el metodo to_sql guardamos el contenido del DataFrame utilizando la ruta a la base de datos que se creo anteriormente
#Solo especificamos el tipo para nombre, de lo contrario se guardara como varchar(max), los demas los guarda
#de acuerdo con el tipo que tiene en el dataframe
catalogo.to_sql(name='Catalogo', 
                con = engine, 
                index=False, 
                if_exists= 'replace',
                dtype= {'nombre' : VARCHAR(length=255)})
#Se especifica que la columna id sea NOT NULL para despues usarla como llave primaria
with engine.connect() as con:
    con.execute('ALTER TABLE Catalogo alter column id int NOT NULL;')
    con.execute('ALTER TABLE Catalogo ADD PRIMARY KEY (id);')


## Guardado de una archivo excel a una tabla

<ul>
    <li>Se leera un archivo de excel</li>
    <li>Se identificara el id del nombre de la estacion. Para esto se hara una consulta a la base de datos</li>
    <li>Se reemplaza la columna "nombre_estacion" por una que tendra el id correspondiente</li>
    <li>Se guardara la tabla en la base de datos</li>
</ul>

In [104]:
#Biblioteca que se usara para hacer match entre palabras
from fuzzywuzzy import process

In [109]:
#Obtenemos las rutas de los archivos xls y creamos un dataframe para guardarlas

#Esta ya no se hace
rutas = []
for i in glob.glob("Puebla/*.xls"):
    rutas.append(i)
datos = {'Rutas' : rutas}
rutas = pd.DataFrame(datos)
rutas.to_csv("Rutas - faltantes.csv")

In [110]:
#Leemos el archivo de rutas que faltan por agregar a la base de datos
rutas = pd.read_csv("Rutas - faltantes.csv")
rutas = rutas.drop(['Unnamed: 0'], axis=1)
rutas.head()

,Rutas
0,Puebla\acapo f13.xls
1,Puebla\huauchi ab14.xls
2,Puebla\huauchi ab15.xls
3,Puebla\huauchi ag14.xls
4,Puebla\huauchi d14.xls


In [111]:
#Leemos el archivo de rutas que faltan por agregar a la base de datos
rutas = pd.read_csv("Rutas - faltantes.csv")
rutas = rutas.drop(['Unnamed: 0'], axis=1)
#Obtenemos la primera ruta y lo eliminamos del DF
r = rutas['Rutas'][0]
rutas = rutas.drop(0, axis=0)
#Se obitiene el catalogo
catalogo = pd.read_sql("SELECT id,nombre from Catalogo", con=engine)
#Se pasa a una lista los nombres de las estaciones del catalogo
listaNombres = catalogo['nombre'].values
#Se abre el archivo de la ruta r
datos = pd.read_excel(r)
#Obtenemos el nombre de la estacion
nombreEstacion = datos["nombre_estacion"][0]
#Buscamos la palabra(de listaNombres) con mas similitud a nombreEstacion y la guardamos en masALto
masAlto, porcentaje = process.extractOne(nombreEstacion,listaNombres)
#Consultamos su id y obtenemos su valor
identificador = (catalogo[catalogo['nombre']==masAlto])['id'].values[0]
#Creamos una columna id_estado donde guardaremos la variable identificador
datos['id_estado'] = datos['nombre_estacion'].map(lambda x: identificador)
#Eliminamos la columna nombre estacion
datos = datos.drop(['nombre_estacion'], axis=1)
#Ordenamos las columnas
columnasOrdenadas = ['id_estado','fecha', 'DirViento', 'DirRafaga', 'RapViento', 'RapRafaga','TempAire', 
                     'HumRelativa', 'PresBarometric', 'Precipitacion', 'RadSolar']
guardar = pd.DataFrame()
guardar[columnasOrdenadas] = datos[columnasOrdenadas]
#Guardamos a la base de datos
guardar.to_sql(name='Datos', con = engine, index=False, if_exists= 'append')


#Se guardan las rutas que faltan por agregar
rutas.to_csv("Rutas - faltantes.csv")

#Si se quiere solo seleccionar algunas columnas para guardarlas en la base de datos se escifican en colBase
#Y el nombre en el archivo de excel en colExcel
#guardar = pd.DataFrame()
#guardar['id_estado'] = datos['nombre_estacion'].map(lambda x: identificador)
#colBase = ['Temperatura','Precipitacion','Humedad']
#colExcel = ['TempAire','Precipitacion','HumRelativa']
#guardar[colBase] = datos[colExcel]



##  Match de palabras

In [99]:
from fuzzywuzzy import process
import pandas as pd

In [100]:
#Se lee el archivo al que se quiere buscar las columnas
df = pd.read_excel("Puebla\huauchi ab14.xls")
df.head()

In [103]:
#Se crea una lista de palabras a buscar
lista = ['HumRelativa', 'TempAire', 'Precipitacion']
#Se obtine el nombre de las columnas en donde se quiere buscar
listaColumnas = df.columns.values
#variable en donde se guardara la palabra con mas parecido
masParecido = []
for i in lista:
    masAlto, porcentaje = process.extractOne(i,listaColumnas)
    masParecido.append(masAlto)
#Se crea un dataFrame en donde solo seleccionamos las columnas cuyo nombre es mas parecido a la lista dada
final = pd.DataFrame()
final[lista] = df[masParecido]
#Mostramos el resultado
final.head()

,HumRelativa,TempAire,Precipitacion
0,70,23.2,0.0
1,72,22.7,0.0
2,73,22.3,0.0
3,74,21.8,0.0
4,75,21.3,0.0
